In [1]:
# Import relevant packages for analysis
import stan as ps
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from patsy import dmatrix, dmatrices
from patsy.contrasts import Treatment, Sum
import arviz as az

# Importing nest_asyncio is only necessary to run pystan in Jupyter Notebooks.
import nest_asyncio
nest_asyncio.apply();

In [3]:
mix = '''data{
  int<lower=0> N;
  int<lower=0> J;
  int<lower=0> M;
  int<lower=0> K;
  matrix[N, J] X;
  matrix[M, K] Z;
  int<lower=0> id[N];
  vector[N] y;
}

parameters {
  matrix[J, M] beta1;
  matrix[K, J] beta2; 
  real<lower=0> tau_ySq;
  vector<lower=0>[J] tau_beta1Sq;
} 

transformed parameters {
  vector[N] y_hat;
  vector[N] y_pred;
  matrix[M, J] mu_beta1;
  mu_beta1 = Z*beta2;
  for (i in 1:N){
    y_hat[i] = X[i,]*beta1[,id[i]];
  }
  for (i in 1:N){
    y_pred[i] = X[i,]*mu_beta1[id[i],]';
  }
}

model {
  real tau_y;
  vector[J] tau_beta1;
  tau_y = sqrt(tau_ySq);
  tau_beta1 = sqrt(tau_beta1Sq);
  y ~ normal(y_hat, tau_y);
  tau_y ~ cauchy(0, 2.5);
  for (i in 1:J){
    beta1[i,] ~ normal(mu_beta1[,i], tau_beta1[i]);
  }
  tau_beta1 ~ cauchy(0, 2.5);
  for (i in 1:J){
    beta2[,i] ~ normal(0, 100);
  }
}
'''

In [4]:
ps.build(mix)

Building...



Building: 228.7s, done.Messages from stanc:
Warning in '/tmp/httpstan_rshg22tp/model_mzotyhox.stan', line 44, column 26: Argument 100 suggests there may be parameters that are not unit scale; consider rescaling with a multiplier (see manual section 22.12).


RuntimeError: Error calling get_param_names: `Exception: variable does not exist; processing stage=data initialization; variable name=N; base type=int (in '/tmp/httpstan_vpnwi9wv/model_mzotyhox.stan', line 2, column 2 to column 17)`